In [1]:

from moviepy.editor import VideoFileClip
from pathlib import Path
import speech_recognition as sr
from pytube import YouTube
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
key = os.getenv('OPENAI_API_KEY')

In [4]:
output_video_path = "G:\Projects\Multimodal-RAG\ideo_data"
output_folder = "mixed_data"
filepath = "ideo_data/input_vid.mp4"
video_url = "https://youtu.be/3dhcmeOTZ_Q"
output_audio_path = "mixed_data/output_audio.wav"

In [12]:
from moviepy.editor import VideoFileClip
import cv2
def video_to_images(input_video, output_folder):
  """Converts all frames of a video to images.

  Args:
    input_video: Path to the input video file.
    output_folder: Path to the output folder where images will be saved.
  """

  cap = cv2.VideoCapture(input_video)

  if not cap.isOpened():
    print("Error opening video file")
    return

  frame_count = 0
  while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
      break

    if frame_count % 100 == 0:
      output_path = f"{output_folder}/frame_{frame_count}.jpg"
      cv2.imwrite(output_path, frame)
    frame_count += 1

  cap.release()
  print("Video frames converted to images successfully!")

In [6]:
def video_to_audio(video_path,output_audio_path):
  clip=VideoFileClip(video_path)
  audio=clip.audio
  audio.write_audiofile(output_audio_path)

In [22]:
def audio_to_text(audio_path):
  recognizer=sr.Recognizer()
  audio=sr.AudioFile(audio_path)

  with audio as source:
    audio_data=recognizer.record(source)

    try:

      #recognize the speech
      text = recognizer.recognize_whisper(audio_data)

    except sr.UnknownValueError:
      print("Speech recognition could not understand the audio.")
  return text

In [8]:

from pytube import YouTube
def download_video(url,output_path):
  yt = YouTube(url)
  metadata = {"Author": yt.author, "Title": yt.title, "Views": yt.views}

  #yt.streams.get_highest_resolution().download(
        #output_path=output_path, filename="input_vid.mp4"
    #)
  return metadata

In [9]:
metadata_vid = download_video(video_url, output_video_path)

In [10]:
metadata_vid

{'Author': '3-Minute Data Science',
 'Title': 'Linear Regression in 3 Minutes',
 'Views': 10274}

In [13]:
video_to_images(filepath, output_folder)

Video frames converted to images successfully!


In [14]:
video_to_audio(filepath, output_audio_path)

MoviePy - Writing audio in mixed_data/output_audio.wav


MoviePy - Done.


In [21]:
output_audio_path 

'mixed_data/output_audio.wav'

In [24]:
text_data = " Lanyard regression is a statistical technique for modeling the relationship between an output variable and one or more input variables. In layman's terms, think of it as fitting a line through some data points as shown here, so you can make predictions on unknown data, assuming there is a linear relationship between the variables. You might be familiar with the linear function y equals mx plus b, where y is the output variable, also called the dependent variable. You may also see expressed as f of x, the function of the input variable. x on the other hand, would serve as the input variable, also called the independent variable. It's likely you'll see the coefficients m and b expressed as beta 1 and beta 0 respectively. So what do the m and b coefficients do? The m or beta 1 coefficient controls the slope of the line. The b or the beta 0 controls the intercept of the line. In machine learning, we also know it as the bias. These two coefficients are what we are solving for in linear regression. We can also extend to multiple input variables, so x1, x2, x3, with beta 1, beta 2, and beta 3, and so on, acting as slopes for each of those variables. In these higher dimensions, you would visualize the linear regression as a hyperplane. So how do we fit the line to these points? Well, you'll notice that there's these differences between the points and the line, these little red segments, these are called residuals. They are the differences between the data points and the predictions the line would produce. Take each of these residuals and square them. These are the squared errors, and notice that the large of the residuals are, the more amplified area of the squares are. If we total the areas of all of these squares for a given line, we will get the sum of the squared error, and this is known as our loss function. We need to find the beta 0 and beta 1 coefficients that will minimize that sum of squared error. The coefficients can be solved with a variety of techniques ranging from matrix decomposition to gradient descent, which is depicted right here. Thankfully, a lot of libraries are available to do this for us, and we will deep dive into these topics in other videos. To validate a linear regression, there are a number of techniques. Machine learning practitioners will often take a third of the data and put it into the test data set. The remaining two thirds will become the training data set. The training data set will then be used to fit the regression line. The test data set will then be used to validate the regression line. This is done to make sure that the regression performs well on data it has not seen before. The tricks used to evaluate the linear regression vary from the R square, standard error of the estimate, prediction intervals, as well as statistical significance. These are topics we will cover in future videos. If you enjoyed this video, please like and subscribe. Look at my two O'Reilly books, Essential Math for Data Science, and getting started with SQL. Chapter five of Essential Math for Data Science actually covers linear regression and much more depth. If you want live instruction, I also do teach on the O'Reilly platform. Promotional link below. I teach classes including machine learning from scratch, probability, and SQL. Comment on what topics you would like to see next, and I will see you again on 3 Minute Data Science."

In [25]:


with open(output_folder + "output_text.txt", "w") as file:
        file.write(text_data)
print("Text data saved to file")
file.close()

Text data saved to file


In [27]:

from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext

In [32]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")



In [34]:
from llama_index.vector_stores.lancedb import LanceDBVectorStore

ModuleNotFoundError: No module named 'numpy.random'

In [ ]:

text_store=LanceDBVectorStore(uri="lancedb",table_name="text_collection")
image_store=LanceDBVectorStore(uri="lancedb",table_name="image_collection")

In [ ]:
storage_context=StorageContext.from_defaults(vector_store=text_store,image_store=image_store)


In [ ]:
documents=SimpleDirectoryReader(output_folder).load_data()


In [ ]:
index = MultiModalVectorStoreIndex.from_documents(documents,storage_context=storage_context)


In [ ]:
retriever_engine=index.as_retriever(similarity_top_k=1, image_similarity_top_k=5)


In [ ]:

from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import ImageNode

In [ ]:
def retrieve(retriever_engine, query_str):
    retrieval_results = retriever_engine.retrieve(query_str)

    retrieved_image = []
    retrieved_text = []
    for res_node in retrieval_results:
        if isinstance(res_node.node, ImageNode):
            retrieved_image.append(res_node.node.metadata["file_path"])
        else:
            display_source_node(res_node, source_length=200)
            retrieved_text.append(res_node.text)

    return retrieved_image, retrieved_text

In [ ]:
query="can you tell me what is linear regression? explain equation of the multiple linear regression?"


In [ ]:
img,text=retrieve(retriever_engine,query)


In [ ]:
import matplotlib.pyplot as plt
def plot_images(images_path):
  images_shown = 0
  plt.figure(figsize=(16, 9))
  for img_path in images_path:
        if os.path.isfile(img_path):
            image = Image.open(img_path)

            plt.subplot(2, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 5:
                break

In [ ]:
plot_images(img)


In [ ]:
qa_tmpl_str=(
    "Based on the provided information, including relevant images and retrieved context from the video, \
    accurately and precisely answer the query without any additional prior knowledge.\n"

    "---------------------\n"
    "Context: {context_str}\n"
    "Metadata for video: {metadata_str} \n"

    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
)

In [ ]:


import json
metadata_str=json.dumps(metadata_vid)

In [ ]:
query_str="can you tell me what is linear regression and equation of linear regression?"


In [ ]:
context_str = "".join(text)


In [ ]:
image_documents = SimpleDirectoryReader( input_files=img).load_data()


In [ ]:
from llama_index.multi_modal_llms.openai import OpenAIMultiModal


In [ ]:
result=openai_mm_llm.complete(
    prompt=qa_tmpl_str.format(
        query_str=query_str,metadata_str=metadata_str
    ),
    image_documents=image_documents,
)

In [ ]:
pprint(result.text)


In [ ]:
qa_tmpl_str=(
    "Based on the provided information, including relevant images and retrieved context from the video, \
    accurately and precisely answer the query without any additional prior knowledge.\n"

    "---------------------\n"
    "Metadata for video: {metadata_str} \n"

    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
)